In [1]:
from datasets import load_dataset
import IPython.display as ipd
from datasets import Dataset, DatasetDict
import torchaudio
import torch
from transformers import WhisperFeatureExtractor
from transformers import WhisperTokenizer
import os

In [2]:
ds = load_dataset("yashtiwari/PaulMooney-Medical-ASR-Data")
ds.column_names
from dotenv import load_dotenv
load_dotenv()
HUGGINGFACE_TOKEN = os.getenv("HUGGINGFACE_TOKEN1")


In [3]:
from huggingface_hub import login

login(token=HUGGINGFACE_TOKEN)


In [4]:
print(ds["train"][0]['sentence'])
ipd.Audio(ds["train"][0]['path']['array'], autoplay=True, rate=44100)


My cough is very heavy and I have mucus.


In [5]:
import torch
import torchaudio

def resample_single_audio(sample):
    # Extract the waveform and sample rate from the audio array
    waveform = torch.tensor(sample["path"]["array"],dtype=torch.float32).unsqueeze(0)  # Add a batch dimension
    sample_rate = sample["path"]["sampling_rate"]
    
    # Resample audio to 16kHz
    resampler = torchaudio.transforms.Resample(orig_freq=sample_rate, new_freq=16000)
    waveform_resampled = resampler(waveform)
    
    # Update the sample with the resampled audio
    sample["path"]["array"] = waveform_resampled.squeeze().numpy().tolist()  # Remove batch dimension and convert to list
    sample["path"]["sampling_rate"] = 16000
    return sample

# Apply resampling to a single audio sample
resampled_audio = resample_single_audio(ds["train"][0])
print(resampled_audio["path"]["sampling_rate"])  # Should print 16000


16000


In [6]:
ipd.Audio(resampled_audio["path"]["array"], autoplay=True, rate=16000)

In [7]:
import torch
from transformers import WhisperProcessor, WhisperForConditionalGeneration
from datasets import load_dataset

# Load model and processor
processor = WhisperProcessor.from_pretrained("openai/whisper-small", language="en")
model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small").to("cuda:6")

# Extract audio sample
sample = resampled_audio["path"]
waveform = torch.tensor(sample["array"], dtype=torch.float32)  # Ensure correct data type

# Process audio
input_features = processor(waveform.numpy(), sampling_rate=sample["sampling_rate"], return_tensors="pt").input_features
# Move input to GPU
input_features = input_features.to("cuda:6")
# Generate token IDs
predicted_ids = model.generate(input_features=input_features)

# Decode token IDs to text
transcription = processor.batch_decode(predicted_ids, skip_special_tokens=True)
print(transcription)


Due to a bug fix in https://github.com/huggingface/transformers/pull/28687 transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English.This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


[' My coach is very heavy and I have muckles.']


In [8]:
def convert_to_dataset(data_list):
    if not data_list:  # Handle empty dataset case
        return Dataset.from_dict({"id": [], "sentence": [], "prompt": [], "speaker_id": [], "audio": []})
    
    ids, sentences, prompts, speaker_ids, audio_data = [], [], [], [], []

    for entry in data_list:
        ids.append(entry["id"])
        sentences.append(entry["sentence"])
        prompts.append(entry["prompt"])
        speaker_ids.append(entry["speaker_id"])

        # Extract preloaded audio correctly
        audio = entry["path"]
        audio_array = audio.get("array", [])
        sampling_rate = audio.get("sampling_rate", 16000)
        
        # Store audio as a dictionary
        audio_data.append({"array": audio_array, "sampling_rate": sampling_rate})

    return Dataset.from_dict({
        "id": ids,
        "sentence": sentences,
        "prompt": prompts,
        "speaker_id": speaker_ids,
        "audio": audio_data
    })

# Assuming ds is already formatted as a dictionary with lists
train_dataset = convert_to_dataset(ds["train"])
val_dataset = convert_to_dataset(ds["validation"])
test_dataset = convert_to_dataset(ds["test"])

# Create DatasetDict
dataset = DatasetDict({
    "train": train_dataset,
    "validation": val_dataset,
    "test": test_dataset
})

print(dataset)

DatasetDict({
    train: Dataset({
        features: ['id', 'sentence', 'prompt', 'speaker_id', 'audio'],
        num_rows: 381
    })
    validation: Dataset({
        features: ['id', 'sentence', 'prompt', 'speaker_id', 'audio'],
        num_rows: 385
    })
    test: Dataset({
        features: ['id', 'sentence', 'prompt', 'speaker_id', 'audio'],
        num_rows: 5895
    })
})


In [9]:
dataset["train"][0]

{'id': '1249120_44160489_107692984',
 'sentence': 'My cough is very heavy and I have mucus.',
 'prompt': 'Cough',
 'speaker_id': 44160489,
 'audio': {'array': [-0.016387939453125,
   -0.015533447265625,
   -0.017333984375,
   -0.016204833984375,
   -0.0155029296875,
   -0.016693115234375,
   -0.016693115234375,
   -0.0184326171875,
   -0.016937255859375,
   -0.013519287109375,
   -0.012908935546875,
   -0.01092529296875,
   -0.011993408203125,
   -0.012542724609375,
   -0.010009765625,
   -0.01251220703125,
   -0.013031005859375,
   -0.010162353515625,
   -0.0125732421875,
   -0.01153564453125,
   -0.00946044921875,
   -0.013031005859375,
   -0.01092529296875,
   -0.010589599609375,
   -0.012725830078125,
   -0.0128173828125,
   -0.01544189453125,
   -0.014312744140625,
   -0.0155029296875,
   -0.018218994140625,
   -0.0167236328125,
   -0.014404296875,
   -0.012298583984375,
   -0.014190673828125,
   -0.015106201171875,
   -0.01373291015625,
   -0.01458740234375,
   -0.0135498046875,


In [11]:
feature_extractor = WhisperFeatureExtractor.from_pretrained("openai/whisper-small")

In [10]:
tokenizer = WhisperTokenizer.from_pretrained("openai/whisper-small", language="English", task="transcribe")

In [12]:
input_str = dataset["train"][0]["sentence"]
labels = tokenizer(input_str).input_ids
decoded_with_special = tokenizer.decode(labels, skip_special_tokens=False)
decoded_str = tokenizer.decode(labels, skip_special_tokens=True)
print(f"Input:                 {input_str}")
print(f"Decoded w/ special:    {decoded_with_special}")
print(f"Decoded w/out special: {decoded_str}")
print(f"Are equal:             {input_str == decoded_str}")

Input:                 My cough is very heavy and I have mucus.
Decoded w/ special:    <|startoftranscript|><|en|><|transcribe|><|notimestamps|>My cough is very heavy and I have mucus.<|endoftext|>
Decoded w/out special: My cough is very heavy and I have mucus.
Are equal:             True


In [9]:
from transformers import WhisperProcessor

processor = WhisperProcessor.from_pretrained("openai/whisper-small",language="English", task="transcribe")

In [14]:
print(dataset["train"][0]["sentence"])

My cough is very heavy and I have mucus.


In [15]:
def resample_audio(batch):
    waveform = torch.tensor(batch["audio"]["array"])  # Convert list to tensor
    sample_rate = batch["audio"]["sampling_rate"]

    # Resample audio to 16kHz
    resampler = torchaudio.transforms.Resample(orig_freq=sample_rate, new_freq=16000)
    waveform_resampled = resampler(waveform)

    # Update batch
    batch["audio"]["array"] = waveform_resampled.numpy().tolist()  # Convert back to list
    batch["audio"]["sampling_rate"] = 16000
    return batch

# Apply resampling
dataset = dataset.map(resample_audio)

Map:   0%|          | 0/381 [00:00<?, ? examples/s]

Map:   0%|          | 0/385 [00:00<?, ? examples/s]

Map:   0%|          | 0/5895 [00:00<?, ? examples/s]

In [16]:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['id', 'sentence', 'prompt', 'speaker_id', 'audio'],
        num_rows: 381
    })
    validation: Dataset({
        features: ['id', 'sentence', 'prompt', 'speaker_id', 'audio'],
        num_rows: 385
    })
    test: Dataset({
        features: ['id', 'sentence', 'prompt', 'speaker_id', 'audio'],
        num_rows: 5895
    })
})


In [17]:
print(dataset["train"][0]["audio"])


{'array': [-0.011181436479091644, -0.01724223792552948, -0.016549458727240562, -0.016197441145777702, -0.011435589753091335, -0.011843937449157238, -0.011829636991024017, -0.01126620639115572, -0.011292035691440105, -0.01279945857822895, -0.0161926057189703, -0.015709534287452698, -0.013505296781659126, -0.014720150269567966, -0.012627040967345238, -0.014939466491341591, -0.01636882685124874, -0.01738543063402176, -0.014645705930888653, -0.018637292087078094, -0.019520215690135956, -0.017286792397499084, -0.01809646561741829, -0.02087930217385292, -0.017923103645443916, -0.017843222245573997, -0.02087314799427986, -0.019636213779449463, -0.022218069061636925, -0.02598143368959427, -0.025395995005965233, -0.026377052068710327, -0.027291901409626007, -0.026540419086813927, -0.02551995776593685, -0.02360270358622074, -0.020945075899362564, -0.021072307601571083, -0.022524962201714516, -0.02240234985947609, -0.022050844505429268, -0.020727118477225304, -0.02091292478144169, -0.018600255250

In [18]:
def prepare_dataset(batch):
    # Initialize lists to store features and labels
    input_features = []
    labels = []

    # Iterate through each item in the batch
    for audio, sentence in zip(batch["audio"], batch["sentence"]):
        # Check if the audio is a dictionary and extract the array directly
        if isinstance(audio, dict) and "array" in audio:
            waveform = torch.tensor(audio["array"]).unsqueeze(0)  # Convert to tensor and add batch dimension
            sample_rate = audio.get("sampling_rate", 16000)  # Default to 16000 if not present
        else:
            # Fallback in case the audio format is unexpected
            waveform, sample_rate = torchaudio.load(audio)

        # Compute log-Mel input features
        features = feature_extractor(waveform.numpy()[0], sampling_rate=sample_rate).input_features[0]
        input_features.append(features)

        # Encode the target text to label ids
        label = tokenizer(sentence).input_ids
        labels.append(label)

    # Assign the list of features and labels to the batch
    batch["input_features"] = input_features
    batch["labels"] = labels
    return batch

common_voice = dataset.map(
    prepare_dataset,
    remove_columns=dataset["train"].column_names,
    batched=True,
    batch_size=16
)


Map:   0%|          | 0/381 [00:00<?, ? examples/s]

Map:   0%|          | 0/385 [00:00<?, ? examples/s]

Map:   0%|          | 0/5895 [00:00<?, ? examples/s]

In [19]:
print(common_voice)

DatasetDict({
    train: Dataset({
        features: ['input_features', 'labels'],
        num_rows: 381
    })
    validation: Dataset({
        features: ['input_features', 'labels'],
        num_rows: 385
    })
    test: Dataset({
        features: ['input_features', 'labels'],
        num_rows: 5895
    })
})


In [4]:
from datasets import DatasetDict

# Save the processed dataset to disk
# common_voice.save_to_disk("common_voice_processed")

# To load it back later
loaded_dataset = DatasetDict.load_from_disk("common_voice_processed")
print(loaded_dataset)

DatasetDict({
    train: Dataset({
        features: ['input_features', 'labels'],
        num_rows: 381
    })
    validation: Dataset({
        features: ['input_features', 'labels'],
        num_rows: 385
    })
    test: Dataset({
        features: ['input_features', 'labels'],
        num_rows: 5895
    })
})


In [5]:
from transformers import WhisperForConditionalGeneration
model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small")
model.to("cuda:6")

WhisperForConditionalGeneration(
  (model): WhisperModel(
    (encoder): WhisperEncoder(
      (conv1): Conv1d(80, 768, kernel_size=(3,), stride=(1,), padding=(1,))
      (conv2): Conv1d(768, 768, kernel_size=(3,), stride=(2,), padding=(1,))
      (embed_positions): Embedding(1500, 768)
      (layers): ModuleList(
        (0-11): 12 x WhisperEncoderLayer(
          (self_attn): WhisperSdpaAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=False)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
        

In [6]:
model.generation_config.language = "English"
model.generation_config.task = "transcribe"

model.generation_config.forced_decoder_ids = None

In [7]:
import torch

from dataclasses import dataclass
from typing import Any, Dict, List, Union

@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any
    decoder_start_token_id: int

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need different padding methods
        # first treat the audio inputs by simply returning torch tensors
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        # get the tokenized label sequences
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        # pad the labels to max length
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        # if bos token is appended in previous tokenization step,
        # cut bos token here as it's append later anyways
        if (labels[:, 0] == self.decoder_start_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch

In [12]:
data_collator = DataCollatorSpeechSeq2SeqWithPadding(
    processor=processor,
    decoder_start_token_id=model.config.decoder_start_token_id,
)

In [13]:
import evaluate

metric = evaluate.load("wer")

In [14]:
def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    # replace -100 with the pad_token_id
    label_ids[label_ids == -100] = tokenizer.pad_token_id

    # we do not want to group tokens when computing the metrics
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    wer = 100 * metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

In [15]:
from transformers import Seq2SeqTrainingArguments
# import accelerate

training_args = Seq2SeqTrainingArguments(
    output_dir="./whisper-small-en",  # change to a repo name of your choice
    per_device_train_batch_size=16,
    gradient_accumulation_steps=1,  # increase by 2x for every 2x decrease in batch size
    learning_rate=1e-5,
    warmup_steps=500,
    max_steps=500,
    gradient_checkpointing=True,
    fp16=True,
    evaluation_strategy="steps",
    per_device_eval_batch_size=8,
    predict_with_generate=True,
    generation_max_length=225,
    save_steps=100,
    eval_steps=100,
    logging_steps=25,
    report_to=["tensorboard"],
    load_best_model_at_end=True,
    metric_for_best_model="wer",
    greater_is_better=False,
    push_to_hub=True,
)

/mnt/nvme_disk2/User_data/nb57077k/miniconda3/envs/ax/lib/python3.10/site-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [17]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=loaded_dataset["train"],
    eval_dataset=loaded_dataset["test"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=processor.feature_extractor,
)

/tmp/ipykernel_2605889/2078443799.py:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [ ]:
trainer.train()

Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.43.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...
/mnt/nvme_disk2/User_data/nb57077k/miniconda3/envs/ax/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Step,Training Loss,Validation Loss,Wer
100,0.170700,0.333353,15.640778


You have passed task=transcribe, but also have set `forced_decoder_ids` to [[1, 50259], [2, 50359], [3, 50363]] which creates a conflict. `forced_decoder_ids` will be ignored in favor of task=transcribe.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
/mnt/nvme_disk2/User_data/nb57077k/miniconda3/envs/ax/lib/python3.10/site-packages/transformers/modeling_utils.py:3353: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 448, 'suppress_tokens': [1, 2, 7, 8, 9, 10, 14, 25, 26, 27, 28, 29, 31, 58, 59, 60, 61, 62, 63, 90, 91, 92, 93, 359, 503, 522, 542, 873, 893, 902, 918, 922, 931, 1350, 1853, 1982, 2460, 2627, 3246, 3253, 3268, 3536, 3846, 3961, 4183, 4667, 6585, 6647, 7273, 9061, 9383, 10428, 10929, 11938, 12033, 12331, 12562, 13793, 14157, 14635, 15265, 15618, 

In [ ]:
ipd.Audio(sample, autoplay=True, rate=16000)